In [17]:
import numpy as np
from tifffile import imread 
import sys
import os
import cv2
from glob import glob
from tqdm import tqdm
import pandas as pd
sys.path.append('../')
from NEATUtils import MovieCreator
from NEATUtils.helpers import save_json, load_json
from NEATModels.TrainConfig import TrainConfig
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path

try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile

In [2]:
###### Specify the image used for making the csv file
SourceDir = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/'

CenterTrainDataDir = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/CenterTrainData/'
GenericTrainDataDir = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/GenericTrainData/'

StaticCenterTrainDataDir = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/StaticCenterTrainData/'
StaticGenericTrainDataDir = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/StaticGenericTrainData/'
Path(StaticCenterTrainDataDir).mkdir(exist_ok = True)
Path(StaticGenericTrainDataDir).mkdir(exist_ok = True)
MovieName = 'EventMovie'
SegMovieName = 'SegEventMovie'
SourceImage = imread(SourceDir + MovieName + '.tif')
SegmentationImage = imread(SourceDir + SegMovieName + '.tif')

NormalCSV = SourceDir + MovieName +'Normal.csv'
ApoptosisCSV = SourceDir + MovieName +'Apoptosis.csv'
DivisionCSV = SourceDir + MovieName +'Division.csv'
MacrocheateCSV = SourceDir + MovieName +'MacroKitty.csv'
NonMatureCSV = SourceDir + MovieName +'Mature.csv'
MatureCSV = SourceDir + MovieName +'Mature.csv'

In [4]:
#X Y Tminus and Tplus for making image volumes
crop_sizeM4 = [124,124,4,5]
crop_sizeM5 = [124,124,5,4]
static_crop_size = [crop_sizeM4[0], crop_sizeM4[1]]
SizeX = crop_sizeM4[0]
SizeY = crop_sizeM4[1]
gridX = 1
gridY = 1
ActionCategories = 3
StaticCategories = 4


In [7]:
ActionEventCSV = [NormalCSV, ApoptosisCSV, DivisionCSV]
StaticEventCSV = [NormalCSV, MacrocheateCSV,NonMatureCSV,MatureCSV]
ActionName = ["Normal", "Apoptosis", "Division"]
StaticName = ["Normal", "Macrocheate", "NonMature", "Mature"]
ActionEventTrain = [0, 1, 2]
StaticEventTrain = [0, 1, 2, 3]
TotalActionCategories = len(ActionEventCSV)
TotalStaticCategories = len(StaticEventCSV)
print('Categories for action training', TotalActionCategories)
print('Categories for static training', TotalStaticCategories)
ActionConfig = TrainConfig(ActionName, ActionEventTrain)
StaticConfig = TrainConfig(StaticName, StaticEventTrain)
Action_json = ActionConfig.to_json()
Static_json = StaticConfig.to_json()
save_json(Action_json, SourceDir + "ActionCategories" + '.json')
save_json(Static_json, SourceDir + "StaticCategories" + '.json')

Categories for action training 3
Categories for static training 4


In [24]:
Json = '/home/kapoorlab/CSVforNeat/NEATcsvfiles/StaticCategories.json'
Readjson = load_json(Json)
for (k, v) in Readjson.items():
   print("Key: " + k)
   print("Value: " + str(v))

Key: Normal
Value: 0
Key: Macrocheate
Value: 1
Key: NonMature
Value: 2
Key: Mature
Value: 3


# For center ONEAT, event is exactly in the center for all training examples

In [ ]:
defname = "MinusFour"
for i in range(TotalActionCategories):
     csv_file = ActionEventCSV[i]
     trainlabel = ActionEventTrain[i]   
     MovieCreator.CreateTrainingMovies(csv_file, SourceImage, SegmentationImage, crop_sizeM4, TotalActionCategories, trainlabel, CenterTrainDataDir, gridX = gridX, gridY = gridY, defname = defname)


In [ ]:
defname = "MinusFive"
for i in range(TotalActionCategories):
     csv_file = ActionEventCSV[i]
     trainlabel = ActionEventTrain[i]   
     MovieCreator.CreateTrainingMovies(csv_file, SourceImage, SegmentationImage, crop_sizeM5, TotalActionCategories, trainlabel, CenterTrainDataDir, gridX = gridX, gridY = gridY, defname = defname)


In [ ]:
for i in range(TotalStaticCategories):
     csv_file = StaticEventCSV[i]
     trainlabel = StaticEventTrain[i] 
     MovieCreator.CreateTrainingImages(csv_file, SourceImage, SegmentationImage, static_crop_size, TotalStaticCategories, trainlabel, StaticCenterTrainDataDir, gridX = gridX, gridY = gridY)   